# Finite Maps
*Finite maps* are structures representing lookup tables `m` over certain key type `k`. Besides the `empty` value, maps support two main operations:
 - `lookup` searches for a value `a` corresponding to given key `k` in a map `m` (this operation may fail which is represented by the `Maybe` functor`)
 - `bind` associates given key `k` to given value `a` in the context of map `m`, producing an updated map

### Implementations
There are several abstract data types which can be used to implement `FiniteMaps`, for instance
 - [*binary search trees*](https://en.wikipedia.org/wiki/Binary_search_tree) will typically yield logarithmic operations (for balanced trees)
 - [*hash tables*](https://en.wikipedia.org/wiki/Hash_table) provide effectively constant `lookup`s but the actual complexity heavily depends on the key hashing function and collision consolidation
 - [*tries*](https://en.wikipedia.org/wiki/Trie) (discussed below) are multi-way prefix trees that take an advantage from key structure with $O(|query|)$ `lookup` and no collisions

In [ ]:
{-# LANGUAGE MultiParamTypeClasses, FlexibleInstances, FlexibleContexts #-}

import Data.Maybe (fromMaybe)

class FiniteMap m k where

    -- | Construct new (empty) map
    empty :: m k a

    -- | Retrieve a value given its key
    lookup :: k -> m k a -> Maybe a

    -- | Bind a value to given key
    bind :: k -> a -> m k a -> m k a

## Trie
*Tries* (also known as *digital search trees*) can be viewed as an extension of *binary search trees* that further exploit the structure of the key type that is used for lookups.

In the implementation presented below, `Trie` is used to bootstrap a `FiniteMap` over a *atomic* key type (*base type*) to a map over an *aggregated* key type. For instance, if the atomic key type is a `Char` then the aggregated type can be a list of characters - i.e. a `String`.

### Representation
The `Trie` representation is similar to the construction of `CatList` or `BootstrapHeap` in that it is a *non-uniform* recursive data type.

The `Trie` data type can be viewed as a multi-way tree with
 - leafs represented by `empty` map `mk` and no values represented by the `Nothing` context
 - nodes with values `a` in the `Just` context and a non-empty map `mk` mapping edge labels to child `Trie`s

More generally, node with `Just a` represents a valid value while `Nothing` represents an invalid node.

Given a `FiniteMap m k`, i.e. a map `m` with keys of the base type `k`, we construct a `FiniteMap` over an aggregated key type `[k]` using a `Trie mk ks a` with
 - the base map type `mk` being `(m k)`, i.e. the structure from the assumed map instance
 - the aggregated key type `ks` being in this case a list of the base type, i.e. `[k]`
 - generic values of type `a`

### Operations
The `lookup` operation is directed by given aggregate key `[k]` and traverses a `Trie`. There are two cases:
 1. The aggregate key has been exhausted -> the result depends on the context of the value `v` in the node that has been reached
 1. During traversal, we first try to `lookup` the base key `k` in the node's map `m` and then follow by a `lookup` of the rest of the aggregate key `ks`. If the former lookup fails (`k` is not in the prefix) then the latter is not tried and `lookup` returns early.

Similar traversal is performed by the `bind` funciton but with the difference that instead of an early return after an unsuccessful `lookup`, a new (`empty`) map is constructed and the traversal (binding) proceeds from this point.

The runtime complexity depends on operations of the base `FiniteMap m k`. However, it is clear that since there cannot be any key collisions in a `Trie`, there upper bound on the number of traversal steps is $O(m)$ where $m$ is the size of the aggregate key (i.e. the query).

### Benefits
As mentioned abote, *tries* can be more efficient than *binary search trees* because the comparison in each node is due to the structure of the key (the map over the base type) more efficient.

Contrary to *hash tables*, there are no key collisions in a *trie*. This means tha in order to `lookup` a value in an imperfect hash table, one might actually need $O(n)$ steps (althought this is typically not the case).

The more interesting benefit of tries over a hash table is that for an unsuccessful `lookup` in the hash table, the whole key must still be processed to compute the hash. This is, however, not the case for tries which in such case can return early (while also keeping successfull searches exact, contrary to e.g. *bloom filters*).

In [ ]:
data Trie mk ks a = Trie (Maybe a) (mk (Trie mk ks a))

instance FiniteMap m k => FiniteMap (Trie (m k)) [k] where

  -- | Construct new (empty) map in O(1) worst-case time
  empty = Trie Nothing empty

  -- | Retrieve a value given its key in O(|query|) worst-case time.
  -- |
  -- | Note: The actual time complexity also depends on 'lookup' of '(m k)'. 
  lookup []       (Trie v m) = v
  lookup (k : ks) (Trie _ m) = lookup k m >>= lookup ks

  -- | Bind a value to given key in O(|query|) worst-case time.
  -- |
  -- | Note: The actual time complexity also depends on 'lookup' and 'bind' of '(m k)'.
  bind [] x (Trie _ m) = Trie (Just x) m
  bind (k : ks) x (Trie v m) =
    let t  = fromMaybe empty (lookup k m)
        t' = bind ks x t
    in  Trie v (bind k t' m)

## Trie of Trees

In [ ]:
data Tree a = E | T a (Tree a) (Tree a)

data Trie mk ks a = Trie (Maybe a) (mk (Trie mk ks (Trie mk ks a)))

instance FiniteMap m k => FiniteMap (Trie (m k)) (Tree k) where

  -- | Construct new (empty) map in O(1) worst-case time
  empty = Trie Nothing empty

  -- | Retrieve a value given its key
  lookup E         (Trie v m) = v
  lookup (T k a b) (Trie _ m) = lookup k m >>= lookup a >>= lookup b

  -- | Bind a value to given key
  bind E x (Trie _ m) = Trie (Just x) m
  bind (T k a b) x (Trie v m) =
    let tt  = fromMaybe empty (lookup k m)
        t   = fromMaybe empty (lookup a tt)
        t'  = bind b x t
        tt' = bind a t' tt
    in  Trie v (bind k tt' m)